# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [42]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [35]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [36]:
spaceship.shape

(8693, 14)

In [37]:
# your code here
spaceship.dropna(inplace=True)

In [38]:
spaceship.shape

(6606, 14)

In [40]:
X_num = spaceship.select_dtypes('number')
standarizer = StandardScaler().fit(X_num)
x_standarized = standarizer.transform(X_num)
df_standarizado = pd.DataFrame(x_standarized, columns=X_num.columns)

X_cat1 = spaceship.select_dtypes('object')
X_cat1.drop(columns=['PassengerId','Name','Cabin'], inplace=True)

X_cat2 = spaceship['Cabin']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_cat2['Cabin'] = le.fit_transform(spaceship['Cabin'])

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoded = encoder.fit_transform(X_cat1).toarray()
unique_values = [encoder.categories_[i] for i in range(len(X_cat1.columns))]
onehot_encoded = pd.DataFrame(encoded, columns=[f"{col}_{value}" for col, values in zip(X_cat1.columns, unique_values) for value in values]) 
X_cat = pd.concat([pd.DataFrame(X_cat2['Cabin']), onehot_encoded], axis=1)
X = pd.concat([X_cat, df_standarizado], axis=1)
X['Cabin'] = X[0]
X.drop(columns=0, inplace=True)
y = spaceship['Transported']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

C:\Users\FX516\AppData\Local\Temp\ipykernel_3780\1672086995.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat2['Cabin'] = le.fit_transform(spaceship['Cabin'])


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [44]:
#your code here
forest = RandomForestRegressor(n_estimators=1000, random_state=0, max_depth=10, max_features='sqrt')
forest.fit(X_train, y_train)   
pred = forest.predict(X_test)


NameError: name 'best_model' is not defined

- Evaluate your model

In [ ]:

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test, y_test))

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [ ]:
#your code here
grid ={"n_estimators": [50, 100, 200],
        "estimator__max_leaf_nodes": [250, 500, None],
        "estimator__max_depth":[10,30]}


- Run Grid Search

In [ ]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)
model.fit(X_train, y_train)
best_model = model.best_estimator_
pred = best_model.predict(X_test)

- Evaluate your model

In [ ]:
# your code here
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test, y_test))  